In [1]:
import numpy as np
import xarray as xr

## Prep array

In [2]:
### make dummy array for QA layer
qa_array = xr.DataArray(np.array(
    [[0, 1, 2, 3, 4, 5, 6, 7, 8, 100, 200, 255],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))
qa_array

<xarray.DataArray (dim_0: 2, dim_1: 12)> Size: 192B
array([[  0,   1,   2,   3,   4,   5,   6,   7,   8, 100, 200, 255],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])
Dimensions without coordinates: dim_0, dim_1

In [3]:
type(qa_array.astype(np.uint8)[0])

xarray.core.dataarray.DataArray

In [7]:
type(qa_array[0])

xarray.core.dataarray.DataArray

In [5]:
### check out numbers in binary form
# bin(0)
bin(255)

'0b11111111'

Make the numbers in the array binary: each number becomes an array of bits (0s and 1s)

In [8]:
### make the array binary
np.unpackbits(
    ### change the "type" of integer in the array
    ### to unsigned 8-bit integer type (values from 0–255) 
    qa_array.astype(np.uint8))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

It gives us 8 bits per number in the original array, which is correct!

But the format is different from what we want.

Add a new axis and unpack the bits in that order:

In [9]:
### make the array binary
np.unpackbits(
    ### change the "type" of integer in the array
    ### to unsigned 8-bit integer type (values from 0–255)
    ### New: keep all the dimensions in the array (currently only 1 dimension)
    ### and add a new axis at the last dimension (we want 2 dimensions)
    qa_array.values.astype(np.uint8)[..., None], 
    
    ### add the axis so that each number in the array is converted
    ### to an array of 8 bits
    axis = -1)

array([[[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 1, 0, 0, 1, 0, 0],
        [1, 1, 0, 0, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]],

       [[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]], dtype=uint8)

This array tells us each number's bit!

## Apply to QA table

Are any of these numbers = 1?

The easiest way to figure this out is to convert the 0's to 1's and vice versa, then multiply, because if there's a 0 (previously a 1) anywhere, the result will be 0.

We can see which QA values we want to keep

In [12]:
### make object for QA array in unsigned 8-bit integer type
qa_uint8_arr = qa_array.values.astype(np.uint8)

### add a dimension, unpack bits into new dimension
qa_bits_arr = np.unpackbits(qa_uint8_arr[..., None],
                            axis = -1)

### select QA values we want to hold onto 
### drop water and snow/ice (4 and 5)
qa_bits_arr = qa_bits_arr[..., [0, 1, 2, 3, 6, 7]]

### identify QA values to mask
np.prod(qa_bits_arr == 0, axis = -1)

### could also do it using sum, then ask if the sum is greater than or equal to 0
### if they are all 0s, this will be true
np.sum(qa_bits_arr, axis = -1) == 0

### make the mask
qa_mask = np.sum(qa_bits_arr, axis = -1) == 0

### apply the mask
qa_array.where(qa_mask)


<xarray.DataArray (dim_0: 2, dim_1: 12)> Size: 192B
array([[ 0., nan, nan, nan,  4., nan, nan, nan,  8., nan, nan, nan],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])
Dimensions without coordinates: dim_0, dim_1

This converts the values we don't want to NaN